<a href="https://colab.research.google.com/github/aim97/Neural-networks/blob/master/CNNs/ResNet/using_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Resnet 50

This demo shows how to use Resnet 50, as well as some considerations needed while writting your code.

In [ ]:
# load necessary libraries
import tensorflow as tf

For the purposes of this demo we will use Resent50 model without pre trained weights, so
```
weights = None
```
We will use MNIST data set for digit recognition, which has 10 classes, in transfer learning process, we usually would remove the top layer at least and add our own, but we will simply incldue the top layer of this model but have 10 classes.
```
include_top=true
```
Since we included the top, we have to tell the model the model what activation to use.
```
classifier_activation='softmax'
```
We have a little problem here, MNIST dataset images are all, 28x28, RESNET input should at least be 32x32, so we will rescale our images to the minimum expected by the model.

In [ ]:
# load trained model
# For now we don't want a trained model we just want the network
image_shape = (32, 32, 1)
model = tf.keras.applications.resnet50.ResNet50(
  # weights='imagenet',
  weights=None,
  include_top=True,
  classes=10,
  classifier_activation='softmax',
  input_shape=image_shape
)

In [ ]:
# load data from keras datasets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [ ]:
# this preporcessing is required by all resent networks and it's easy to apply.
def preprocess(imgs, image_shape):
  # apply resent preprocessing
  shape = imgs.shape
  x = tf.keras.applications.resnet50.preprocess_input(imgs)
  # add an additional channel
  x = x.reshape(shape + (1, ))
  # resize the image
  x = tf.image.resize(x, image_shape[:-1])
  # normalize image pixels
  x /= 255.0
  return x

print(x_train.shape)
print(x_test.shape)

x_train = preprocess(x_train, image_shape)
x_test = preprocess(x_test, image_shape)
# x_train = tf.image.resize(preprocess(x_train).reshape(x_train.shape + (1,) ), image_shape[:-1]) / 255.0
# x_test  = tf.image.resize(preprocess(x_test).reshape(x_test.shape + (1,) ), image_shape[:-1]) / 255.0

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 32, 32, 1)
(10000, 32, 32, 1)


From this point onwards we proceed normally, we compile the model, then train it on the data we have.

In [ ]:
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

# Danger
If you try to run this model it will take a long time for training and won't reach a good result on this problem:

## Why

1. The simplicity of Digit recognition, so a small model will work fine on it. (we only use MNIST since it's just a demo).
2. **Resnet50 has a large number of layers**, too many parameters to train, so it naturally takes long regardless of the application (it depends on the machine of course), so *you should only try a large network once you decided that you actually need one*, otherwise doing so would be a waste of time.
3. After running the model for about two hours, it only finished 2 epochs with accurcy 96%, which is close but not better than smaller models, which makes sense, Resnet is made to make sure adding more layers doesn't hurt performance but it doesn't necerssarily improve it.

In [12]:
model.fit(x=x_train, y=y_train, batch_size=32, epochs=4)

Epoch 1/4
1875/1875 [==============================] - 3325s 2s/step - loss: 0.4113 - accuracy: 0.9072
Epoch 2/4
1875/1875 [==============================] - 4562s 2s/step - loss: 0.1561 - accuracy: 0.9648
Epoch 3/4
1875/1875 [==============================] - 4399s 2s/step - loss: 0.2320 - accuracy: 0.9452
Epoch 4/4
1875/1875 [==============================] - 4304s 2s/step - loss: 0.1550 - accuracy: 0.9620
